# Fine-tuning your XGBoost model
>  This chapter will teach you how to make your XGBoost models as performant as possible. You'll learn about the variety of parameters that can be adjusted to alter the behavior of XGBoost and how to tune them efficiently so that you can supercharge the performance of your models.

- toc: true 
- badges: true
- comments: true
- author: Lucas Nunes
- categories: [Datacamp, XGBoost]
- image: images/datacamp/___

> Note: This is a summary of the course's chapter 3 exercises "Extreme Gradient Boosting with XGBoost" at datacamp. <br>[Github repo](https://github.com/lnunesAI/Datacamp/) / [Course link](https://www.datacamp.com/tracks/machine-learning-scientist-with-python)

In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import xgboost as xgb

plt.rcParams['figure.figsize'] = (8, 8)

## Why tune your model?

### When is tuning your model a bad idea?

<p>Now that you've seen the effect that tuning has on the overall performance of your XGBoost model, let's turn the question on its head and see if you can figure out when tuning your model might not be the best idea. <strong>Given that model tuning can be time-intensive and complicated, which of the following scenarios would NOT call for careful tuning of your model</strong>?</p>

<pre>
Possible Answers

You have lots of examples from some dataset and very many features at your disposal.

<b>You are very short on time before you must push an initial model to production and have little data to train your model on.</b>

You have access to a multi-core (64 cores) server with lots of memory (200GB RAM) and no time constraints.

You must squeeze out every last bit of performance out of your xgboost model.

</pre>

**You cannot tune if you do not have time!**

### Tuning the number of boosting rounds

<div class=""><p>Let's start with parameter tuning by seeing how the number of boosting rounds (number of trees you build) impacts the out-of-sample performance of your XGBoost model. You'll use <code>xgb.cv()</code> inside a <code>for</code> loop and build one model per <code>num_boost_round</code> parameter.</p>
<p>Here, you'll continue working with the Ames housing dataset. The features are available in the array <code>X</code>, and the target vector is contained in <code>y</code>.</p></div>

In [2]:
df = pd.read_csv('https://raw.githubusercontent.com/lnunesAI/Datacamp/main/2-machine-learning-scientist-with-python/05-extreme-gradient-boosting-with-xgboost/datasets/boston_1460x57.csv')

In [3]:
X, y = df.iloc[:, :-1], df.iloc[:, -1]

Instructions
<ul>
<li>Create a <code>DMatrix</code> called <code>housing_dmatrix</code> from <code>X</code> and <code>y</code>.</li>
<li>Create a parameter dictionary called <code>params</code>, passing in the appropriate <code>"objective"</code> (<code>"reg:linear"</code>) and <code>"max_depth"</code> (set it to <code>3</code>).</li>
<li>Iterate over <code>num_rounds</code> inside a <code>for</code> loop and perform 3-fold cross-validation. In each iteration of the loop, pass in the current number of boosting rounds (<code>curr_num_rounds</code>) to <code>xgb.cv()</code> as the argument to <code>num_boost_round</code>. </li>
<li>Append the final boosting round RMSE for each cross-validated XGBoost model to the <code>final_rmse_per_round</code> list.</li>
<li><code>num_rounds</code> and <code>final_rmse_per_round</code> have been zipped and converted into a DataFrame so you can easily see how the model performs with each boosting round. Hit 'Submit Answer' to see the results!</li>
</ul>

In [8]:
# Create the DMatrix: housing_dmatrix
housing_dmatrix = xgb.DMatrix(X, y)

# Create the parameter dictionary for each tree: params 
params = {"objective":"reg:squarederror", "max_depth":3} #reg:linear

# Create list of number of boosting rounds
num_rounds = [5, 10, 15]

# Empty list to store final round rmse per XGBoost model
final_rmse_per_round = []

# Iterate over num_rounds and build one model per num_boost_round parameter
for curr_num_rounds in num_rounds:

    # Perform cross-validation: cv_results
    cv_results = xgb.cv(dtrain=housing_dmatrix, params=params, nfold=3, num_boost_round=curr_num_rounds, metrics="rmse", as_pandas=True, seed=123)
    
    # Append final round RMSE
    final_rmse_per_round.append(cv_results["test-rmse-mean"].tail().values[-1])

# Print the resultant DataFrame
num_rounds_rmses = list(zip(num_rounds, final_rmse_per_round))
print(pd.DataFrame(num_rounds_rmses,columns=["num_boosting_rounds","rmse"]))

   num_boosting_rounds          rmse
0                    5  50903.299479
1                   10  34774.195313
2                   15  32895.096354


**As you can see, increasing the number of boosting rounds decreases the RMSE.**

### Automated boosting round selection using early_stopping

<div class=""><p>Now, instead of attempting to cherry pick the best possible number of boosting rounds, you can very easily have XGBoost automatically select the number of boosting rounds for you within <code>xgb.cv()</code>. This is done using a technique called <strong>early stopping</strong>. </p>
<p><strong>Early stopping</strong> works by testing the XGBoost model after every boosting round against a hold-out dataset and stopping the creation of additional boosting rounds (thereby finishing training of the model early) if the hold-out metric (<code>"rmse"</code> in our case) does not improve for a given number of rounds. Here you will use the <code>early_stopping_rounds</code> parameter in <code>xgb.cv()</code> with a large possible number of boosting rounds (50). Bear in mind that if the holdout metric continuously improves up through when <code>num_boost_rounds</code> is reached, then early stopping does not occur.</p>
<p>Here, the <code>DMatrix</code> and parameter dictionary have been created for you. Your task is to use cross-validation with early stopping. Go for it!</p></div>

Instructions
<ul>
<li>Perform 3-fold cross-validation with early stopping and <code>"rmse"</code> as your metric. Use <code>10</code> early stopping rounds and <code>50</code> boosting rounds. Specify a <code>seed</code> of <code>123</code> and make sure the output is a <code>pandas</code> DataFrame. Remember to specify the other parameters such as <code>dtrain</code>, <code>params</code>, and <code>metrics</code>.</li>
<li>Print <code>cv_results</code>.</li>
</ul>

In [10]:
# Create your housing DMatrix: housing_dmatrix
housing_dmatrix = xgb.DMatrix(data=X, label=y)

# Create the parameter dictionary for each tree: params
params = {"objective":"reg:squarederror", "max_depth":4}

# Perform cross-validation with early stopping: cv_results
cv_results = xgb.cv(dtrain=housing_dmatrix, params=params, nfold=3, early_stopping_rounds=10, num_boost_round=50, metrics="rmse", as_pandas=True, seed=123)
    

# Print cv_results
print(cv_results)

    train-rmse-mean  train-rmse-std  test-rmse-mean  test-rmse-std
0     141871.619792      403.625044   142640.651042     705.559164
1     103057.020833       73.773330   104907.661458     111.116005
2      75975.960937      253.731460    79262.057292     563.761707
3      57420.535156      521.648794    61620.139323    1087.688565
4      44552.957031      544.167830    50437.565104    1846.445052
5      35763.953125      681.798925    43035.660156    2034.472841
6      29861.465495      769.569137    38600.881510    2169.801278
7      25994.675781      756.523981    36071.817708    2109.795546
8      23306.836589      759.236242    34383.184896    1934.543836
9      21459.768880      745.624686    33509.141276    1887.375284
10     20148.722656      749.611655    32916.805990    1850.898680
11     19215.381510      641.388117    32197.832682    1734.456935
12     18627.388672      716.256238    31770.852213    1802.153746
13     17960.694010      557.041305    31482.781250    1779.12

### Overview of XGBoost's hyperparameters

### Tuning eta

<div class=""><p>It's time to practice tuning other XGBoost hyperparameters in earnest and observing their effect on model performance! You'll begin by tuning the <code>"eta"</code>, also known as the learning rate.</p>
<p>The learning rate in XGBoost is a parameter that can range between <code>0</code> and <code>1</code>, with higher values of <code>"eta"</code> penalizing feature weights more strongly, causing much stronger regularization.</p></div>

Instructions
<ul>
<li>Create a list called <code>eta_vals</code> to store the following <code>"eta"</code> values: <code>0.001</code>, <code>0.01</code>, and <code>0.1</code>.</li>
<li>Iterate over your <code>eta_vals</code> list using a <code>for</code> loop.</li>
<li>In each iteration of the <code>for</code> loop, set the <code>"eta"</code> key of <code>params</code> to be equal to <code>curr_val</code>. Then, perform 3-fold cross-validation with early stopping (<code>5</code> rounds), <code>10</code> boosting rounds, a metric of <code>"rmse"</code>, and a <code>seed</code> of <code>123</code>. Ensure the output is a DataFrame.</li>
<li>Append the final round RMSE to the <code>best_rmse</code> list.</li>
</ul>

In [12]:
# Create your housing DMatrix: housing_dmatrix
housing_dmatrix = xgb.DMatrix(data=X, label=y)

# Create the parameter dictionary for each tree (boosting round)
params = {"objective":"reg:squarederror", "max_depth":3}

# Create list of eta values and empty list to store final round rmse per xgboost model
eta_vals = [0.001, 0.01, 0.1]
best_rmse = []

# Systematically vary the eta 
for curr_val in eta_vals:

    params["eta"] = curr_val
    
    # Perform cross-validation: cv_results
    cv_results = cv_results = xgb.cv(dtrain=housing_dmatrix, params=params, nfold=3, early_stopping_rounds=5, num_boost_round=10, metrics="rmse", as_pandas=True, seed=123)
    
    
    
    
    # Append the final round rmse to best_rmse
    best_rmse.append(cv_results["test-rmse-mean"].tail().values[-1])

# Print the resultant DataFrame
print(pd.DataFrame(list(zip(eta_vals, best_rmse)), columns=["eta","best_rmse"]))

     eta      best_rmse
0  0.001  195736.416667
1  0.010  179932.161458
2  0.100   79759.416667


### Tuning max_depth

<p>In this exercise, your job is to tune <code>max_depth</code>, which is the parameter that dictates the maximum depth that each tree in a boosting round can grow to. Smaller values will lead to shallower trees, and larger values to deeper trees.</p>

Instructions
<ul>
<li>Create a list called <code>max_depths</code> to store the following <code>"max_depth"</code> values: <code>2</code>, <code>5</code>, <code>10</code>, and <code>20</code>.</li>
<li>Iterate over your <code>max_depths</code> list using a <code>for</code> loop.</li>
<li>Systematically vary <code>"max_depth"</code> in each iteration of the <code>for</code> loop and perform 2-fold cross-validation with early stopping (<code>5</code> rounds), <code>10</code> boosting rounds, a metric of <code>"rmse"</code>, and a <code>seed</code> of <code>123</code>. Ensure the output is a DataFrame.</li>
</ul>

In [14]:
# Create your housing DMatrix
housing_dmatrix = xgb.DMatrix(data=X,label=y)

# Create the parameter dictionary
params = {"objective":"reg:squarederror"}

# Create list of max_depth values
max_depths = [2, 5, 10, 20]
best_rmse = []

# Systematically vary the max_depth
for curr_val in max_depths:

    params["max_depth"] = curr_val
    
    # Perform cross-validation
    cv_results = xgb.cv(dtrain=housing_dmatrix, params=params, nfold=2, early_stopping_rounds=5, num_boost_round=10, metrics="rmse", as_pandas=True, seed=123)
    
    
    
    
    # Append the final round rmse to best_rmse
    best_rmse.append(cv_results["test-rmse-mean"].tail().values[-1])

# Print the resultant DataFrame
print(pd.DataFrame(list(zip(max_depths, best_rmse)),columns=["max_depth","best_rmse"]))

   max_depth     best_rmse
0          2  37957.472657
1          5  35596.595703
2         10  36065.548829
3         20  36739.576172


### Tuning colsample_bytree

<p>Now, it's time to tune <code>"colsample_bytree"</code>. You've already seen this if you've ever worked with scikit-learn's <code>RandomForestClassifier</code> or <code>RandomForestRegressor</code>, where it just was called <code>max_features</code>. In both <code>xgboost</code> and <code>sklearn</code>, this parameter (although named differently) simply specifies the fraction of features to choose from at every split in a given tree. In <code>xgboost</code>, <code>colsample_bytree</code> must be specified as a float between 0 and 1.</p>

Instructions
<ul>
<li>Create a list called <code>colsample_bytree_vals</code> to store the values <code>0.1</code>, <code>0.5</code>, <code>0.8</code>, and <code>1</code>.</li>
<li>Systematically vary <code>"colsample_bytree"</code> and perform cross-validation, exactly as you did with <code>max_depth</code> and <code>eta</code> previously.</li>
</ul>

In [16]:
# Create your housing DMatrix
housing_dmatrix = xgb.DMatrix(data=X,label=y)

# Create the parameter dictionary
params={"objective":"reg:squarederror","max_depth":3}

# Create list of hyperparameter values: colsample_bytree_vals
colsample_bytree_vals = [0.1, 0.5, 0.8, 1]
best_rmse = []

# Systematically vary the hyperparameter value 
for curr_val in colsample_bytree_vals:

    params["colsample_bytree"] = curr_val
    
    # Perform cross-validation
    cv_results = xgb.cv(dtrain=housing_dmatrix, params=params, nfold=2,
                 num_boost_round=10, early_stopping_rounds=5,
                 metrics="rmse", as_pandas=True, seed=123)
    
    # Append the final round rmse to best_rmse
    best_rmse.append(cv_results["test-rmse-mean"].tail().values[-1])

# Print the resultant DataFrame
print(pd.DataFrame(list(zip(colsample_bytree_vals, best_rmse)), columns=["colsample_bytree","best_rmse"]))

   colsample_bytree     best_rmse
0               0.1  48193.453125
1               0.5  36013.542969
2               0.8  35932.960938
3               1.0  35836.044922


**There are several other individual parameters that you can tune, such as "subsample", which dictates the fraction of the training data that is used during any given boosting round. Next up: Grid Search and Random Search to tune XGBoost hyperparameters more efficiently!**

## Review of grid search and random search

### Grid search with XGBoost

<p>Now that you've learned how to tune parameters individually with XGBoost, let's take your parameter tuning to the next level by using scikit-learn's <code>GridSearch</code> and <code>RandomizedSearch</code> capabilities with internal cross-validation using the <code>GridSearchCV</code> and <code>RandomizedSearchCV</code> functions. You will use these to find the best model exhaustively from a collection of possible parameter values across multiple parameters simultaneously. Let's get to work, starting with <code>GridSearchCV</code>!</p>

In [18]:
from sklearn.model_selection import GridSearchCV

Instructions
<ul>
<li>Create a parameter grid called <code>gbm_param_grid</code> that contains a list of <code>"colsample_bytree"</code> values (<code>0.3</code>, <code>0.7</code>), a list with a single value for <code>"n_estimators"</code> (<code>50</code>), and a list of 2 <code>"max_depth"</code> (<code>2</code>, <code>5</code>) values.</li>
<li>Instantiate an <code>XGBRegressor</code> object called <code>gbm</code>.</li>
<li>Create a <code>GridSearchCV</code> object called <code>grid_mse</code>, passing in: the parameter grid to <code>param_grid</code>, the <code>XGBRegressor</code> to <code>estimator</code>, <code>"neg_mean_squared_error"</code> to <code>scoring</code>, and <code>4</code> to <code>cv</code>. Also specify <code>verbose=1</code> so you can better understand the output.</li>
<li>Fit the <code>GridSearchCV</code> object to <code>X</code> and <code>y</code>.</li>
<li>Print the best parameter values and lowest RMSE, using the <code>.best_params_</code> and <code>.best_score_</code> attributes, respectively, of <code>grid_mse</code>.</li>
</ul>

In [20]:
gbm

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0,
             importance_type='gain', learning_rate=0.1, max_delta_step=0,
             max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
             n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
             silent=None, subsample=1, verbosity=1)

In [29]:
# Create the parameter grid: gbm_param_grid
gbm_param_grid = {
    'colsample_bytree': [0.3, 0.7],
    'n_estimators': [50],
    'max_depth': [2, 5]
}

# Instantiate the regressor: gbm
gbm = xgb.XGBRegressor(objective='reg:squarederror')

# Perform grid search: grid_mse
grid_mse = GridSearchCV(estimator=gbm, param_grid=gbm_param_grid,
scoring='neg_mean_squared_error', cv=4, verbose=1, n_jobs=-1)

# Fit grid_mse to the data
grid_mse.fit(X, y)

# Print the best parameters and lowest RMSE
print("Best parameters found: ", grid_mse.best_params_)
print("Lowest RMSE found: ", np.sqrt(np.abs(grid_mse.best_score_)))

Fitting 4 folds for each of 4 candidates, totalling 16 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.


Best parameters found:  {'colsample_bytree': 0.7, 'max_depth': 5, 'n_estimators': 50}
Lowest RMSE found:  29916.562522854438


[Parallel(n_jobs=-1)]: Done  16 out of  16 | elapsed:    1.0s finished


### Random search with XGBoost

<p>Often, <code>GridSearchCV</code> can be really time consuming, so in practice, you may want to use <code>RandomizedSearchCV</code> instead, as you will do in this exercise. The good news is you only have to make a few modifications to your <code>GridSearchCV</code> code to do <code>RandomizedSearchCV</code>. The key difference is you have to specify a <code>param_distributions</code> parameter instead of a <code>param_grid</code> parameter.</p>

In [31]:
from sklearn.model_selection import RandomizedSearchCV

Instructions
<ul>
<li>Create a parameter grid called <code>gbm_param_grid</code> that contains a list with a single value for <code>'n_estimators'</code> (<code>25</code>), and a list of <code>'max_depth'</code> values between <code>2</code> and <code>11</code> for <code>'max_depth'</code> - use <code>range(2, 12)</code> for this. </li>
<li>Create a <code>RandomizedSearchCV</code> object called <code>randomized_mse</code>, passing in: the parameter grid to <code>param_distributions</code>, the <code>XGBRegressor</code> to <code>estimator</code>, <code>"neg_mean_squared_error"</code> to <code>scoring</code>, <code>5</code> to <code>n_iter</code>, and <code>4</code> to <code>cv</code>. Also specify <code>verbose=1</code> so you can better understand the output.</li>
<li>Fit the <code>RandomizedSearchCV</code> object to <code>X</code> and <code>y</code>.</li>
</ul>

In [35]:
# Create the parameter grid: gbm_param_grid 
gbm_param_grid = {
    'n_estimators': [25],
    'max_depth': range(2, 12)
}

# Instantiate the regressor: gbm
gbm = xgb.XGBRegressor(objective='reg:squarederror', n_estimators=10)

# Perform random search: grid_mse
randomized_mse = RandomizedSearchCV(estimator=gbm,param_distributions=gbm_param_grid, n_iter=5, scoring='neg_mean_squared_error', cv=4, verbose=1, n_jobs=-1)


# Fit randomized_mse to the data
randomized_mse.fit(X,y)

# Print the best parameters and lowest RMSE
print("Best parameters found: ", randomized_mse.best_params_)
print("Lowest RMSE found: ", np.sqrt(np.abs(randomized_mse.best_score_)))

Fitting 4 folds for each of 5 candidates, totalling 20 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.


Best parameters found:  {'n_estimators': 25, 'max_depth': 8}
Lowest RMSE found:  37418.52437954323


[Parallel(n_jobs=-1)]: Done  20 out of  20 | elapsed:    1.5s finished


## Limits of grid search and random search

### When should you use grid search and random search?

<p>Now that you've seen some of the drawbacks of grid search and random search, which of the following most accurately describes why both random search and grid search are non-ideal search hyperparameter tuning strategies in all scenarios?</p>

<pre>
Possible Answers

Grid Search and Random Search both take a very long time to perform, regardless of the number of parameters you want to tune.

Grid Search and Random Search both scale exponentially in the number of hyperparameters you want to tune.

<b>The search space size can be massive for Grid Search in certain cases, whereas for Random Search the number of hyperparameters has a significant effect on how long it takes to run.</b>

Grid Search and Random Search require that you have some idea of where the ideal values for hyperparameters reside.
</pre>

**This is why random search and grid search should not always be used.**